In [1]:
import tensorflow as tf
import numpy as np
import os

In [2]:
print(tf.__version__)

1.10.0


In [3]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
# Where to save the figures
PROJECT_ROOT_DIR = os.getcwd()
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    
def save_model(path_string):
    return os.path.join(PROJECT_ROOT_DIR, path_string)

In [4]:
os.getcwd()

'C:\\Users\\Administrator\\hands-on-ml-with-sklearn-and-tf'

In [5]:
save_model("tmp/my_model.ckpt")

'C:\\Users\\Administrator\\hands-on-ml-with-sklearn-and-tf\\tmp/my_model.ckpt'

In [6]:
x = tf.Variable(3,name="x")
y = tf.Variable(4,name="y")
f = x*x*y + y+2

In [7]:
#way1
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

print(result)
sess.close()

42


In [8]:
#way2
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

42


In [9]:
#way3
init =  tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


In [10]:
#way4
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [11]:
x1 = tf.Variable(6)
x1.graph is tf.get_default_graph()

True

In [12]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(7)
x2.graph is graph


True

In [13]:
x2.graph is tf.get_default_graph()

False

In [14]:
tf.reset_default_graph()

# Linear Regression with Tensorflow

In [15]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m,n = housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

X = tf.constant(housing_data_plus_bias,dtype = tf.float32,name = "X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")

XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value = theta.eval()
print(theta_value)
                    

[[-3.7465141e+01]
 [ 4.3573415e-01]
 [ 9.3382923e-03]
 [-1.0662201e-01]
 [ 6.4410698e-01]
 [-4.2513184e-06]
 [-3.7732250e-03]
 [-4.2664889e-01]
 [-4.4051403e-01]]


$$\hat{\theta}= \left ( X^{T}\cdot X \right )^{-1}\cdot X^{T}\cdot y$$

Compared with numpy

In [16]:
X = housing_data_plus_bias
y = housing.target.reshape(-1,1)

theta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y) 

In [17]:
theta

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

Compared with sklearn

In [18]:
from sklearn.linear_model import LinearRegression

X = housing.data
y = housing.target.reshape(-1,1)
linear = LinearRegression()
linear.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [19]:
np.r_[linear.intercept_.reshape(-1,1),linear.coef_.reshape(-1,1)]

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

$$MSE = \frac{1}{m}\sum_{i=1}^{m}(y_{i}-\hat{y_{i}})^{2}$$

In [20]:
from sklearn.preprocessing import  StandardScaler

learning_rate = 0.001
n_epoch = 1000

housing = fetch_california_housing()

m,n = housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]
#y = housing.target.reshape(-1,1)
scale_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)

X = tf.constant(scale_housing_data_plus_bias,dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1,1),dtype=tf.float32)
y_pred = tf.matmul(X,theta,name="prediction")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")

#使用TensorFlow自动梯度计算
#gradients = 2/m * tf.matmul(tf.transpose(X),error)
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta,theta - learning_rate*gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch%100==0:
            print("Epoch = ",epoch,"MSE = ",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch =  0 MSE =  8.606744
Epoch =  100 MSE =  6.8969636
Epoch =  200 MSE =  6.0291204
Epoch =  300 MSE =  5.5771976
Epoch =  400 MSE =  5.3342395
Epoch =  500 MSE =  5.1985183
Epoch =  600 MSE =  5.1192565
Epoch =  700 MSE =  5.0706162
Epoch =  800 MSE =  5.0391335
Epoch =  900 MSE =  5.017601


In [21]:
best_theta

array([[-0.36589313],
       [ 0.68926954],
       [ 0.30235562],
       [ 0.23903574],
       [-0.18268274],
       [ 0.10795614],
       [ 0.04576566],
       [ 0.02747855],
       [ 0.10496338]], dtype=float32)

# 使用优化器

In [22]:
reset_graph()

X = tf.constant(scale_housing_data_plus_bias,dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1,1),dtype=tf.float32)
y_pred = tf.matmul(X,theta,name="prediction")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")

#使用TensorFlow自动梯度计算
#gradients = 2/m * tf.matmul(tf.transpose(X),error)
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta,theta - learning_rate*gradients)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse) 

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        if epoch%100==0:
            print("Epoch = ",epoch,"MSE = ",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch =  0 MSE =  7.5898657
Epoch =  100 MSE =  6.4475336
Epoch =  200 MSE =  5.82356
Epoch =  300 MSE =  5.465709
Epoch =  400 MSE =  5.2503314
Epoch =  500 MSE =  5.1148367
Epoch =  600 MSE =  5.026291
Epoch =  700 MSE =  4.966593
Epoch =  800 MSE =  4.925329
Epoch =  900 MSE =  4.896237


In [23]:
reset_graph()

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1,1),dtype=tf.float32)
y_pred = tf.matmul(X,theta,name="prediction")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse) 

def fetch_batch(epoch,batch_index,batch_size):
    np.random.seed(epoch*n_batches+batch_size)
    indices = np.random.randint(m,size=batch_size)
    X_batch = scale_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return X_batch,y_batch
    
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        for batch_index in range(n_batches):
            X_batch,y_batch = fetch_batch(epoch,batch_size,batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta = theta.eval()

In [24]:
reset_graph()

n_epoch = 1000

X = tf.constant(scale_housing_data_plus_bias,dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1,1),dtype=tf.float32)
y_pred = tf.matmul(X,theta,name="prediction")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")

#使用TensorFlow自动梯度计算
#gradients = 2/m * tf.matmul(tf.transpose(X),error)
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta,theta - learning_rate*gradients)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse) 

init = tf.global_variables_initializer()
saver = tf.train.Saver()



In [25]:
with tf.Session() as sess:
    saver.restore(sess, "tmp/my_model_final.ckpt")#注意删除 sess.run(init)
    for epoch in range(n_epoch):
        if epoch % 100 == 0: # checkpoint every 100 epochs
            save_path = saver.save(sess, save_model("tmp/my_model.ckpt"))
            print("Epoch = ",epoch,"MSE = ",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, save_model("tmp/my_model_final.ckpt"))

INFO:tensorflow:Restoring parameters from tmp/my_model_final.ckpt
Epoch =  0 MSE =  4.8064866
Epoch =  100 MSE =  4.8063827
Epoch =  200 MSE =  4.8062825
Epoch =  300 MSE =  4.806185
Epoch =  400 MSE =  4.806091
Epoch =  500 MSE =  4.806
Epoch =  600 MSE =  4.8059115
Epoch =  700 MSE =  4.8058267
Epoch =  800 MSE =  4.8057437
Epoch =  900 MSE =  4.805664


In [26]:
reset_graph()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)


X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1,1),dtype=tf.float32)
y_pred = tf.matmul(X,theta,name="prediction")
error = y_pred-y
mse = tf.reduce_mean(tf.square(error),name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse) 

def fetch_batch(epoch,batch_index,batch_size):
    np.random.seed(epoch*n_batches+batch_size)
    indices = np.random.randint(m,size=batch_size)
    X_batch = scale_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return X_batch,y_batch
    
    
init = tf.global_variables_initializer()
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        for batch_index in range(n_batches):
            X_batch,y_batch = fetch_batch(epoch,batch_size,batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta = theta.eval()
file_writer.close()
print(best_theta)


[[-0.94757867]
 [ 0.87069815]
 [ 0.23070854]
 [-0.07395332]
 [ 0.24779524]
 [ 0.18587995]
 [-1.1587862 ]
 [-0.88808215]
 [-0.84004384]]


In [27]:
reset_graph()

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)


X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1,1],-1,1),dtype=tf.float32)
y_pred = tf.matmul(X,theta,name="prediction")
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")


optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse) 

def fetch_batch(epoch,batch_index,batch_size):
    np.random.seed(epoch*n_batches+batch_size)
    indices = np.random.randint(m,size=batch_size)
    X_batch = scale_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1,1)[indices]
    return X_batch,y_batch
    
    
init = tf.global_variables_initializer()
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        for batch_index in range(n_batches):
            X_batch,y_batch = fetch_batch(epoch,batch_size,batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta = theta.eval()
file_writer.close()
print(best_theta)


[[-0.94757867]
 [ 0.87069815]
 [ 0.23070854]
 [-0.07395332]
 [ 0.24779524]
 [ 0.18587995]
 [-1.1587862 ]
 [-0.88808215]
 [-0.84004384]]


# 模块性

In [29]:
reset_graph()

n_features = 3
X = tf.placeholder(tf.float32,shape=(None,n_features),name="X")
w1 = tf.Variable(tf.random_normal((n_features,1)),name="w1")
w2 = tf.Variable(tf.random_normal((n_features,1)),name="w2")
b1 = tf.Variable(0.0,name="bias1")
b2 = tf.Variable(0.0,name="bias2")
z1 = tf.add(tf.matmul(X,w1),b1,name="z1")
z2 = tf.add(tf.matmul(X,w2),b2,name="z2")
relu1 = tf.maximum(z1,0,name="relu1")
relu2 = tf.maximum(z2,0,name="relu2")
output = tf.add(relu1,relu2,name="output")


file_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

In [31]:
reset_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(X,w),b,name="z")
    return tf.maximum(z,0.0,name="relu")

n_features = 3
X = tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus = [relu(X) for i in range(5)] 
output = tf.add_n(relus,name="output")

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())

In [32]:
reset_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]),1)
        w = tf.Variable(tf.random_normal(w_shape),name="weights")
        b = tf.Variable(0.0,name="bias")
        z = tf.add(tf.matmul(X,w),b,name="z")
        return tf.maximum(z,0.0,name="relu")

n_features = 3
X = tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus = [relu(X) for i in range(5)] 
output = tf.add_n(relus,name="output")

file_writer = tf.summary.FileWriter("logs/relu3", tf.get_default_graph())

In [34]:
reset_graph()

def relu(X):
    threshold = tf.get_variable("threshold",shape=(),initializer=tf.constant_initializer(0.0))
    w_shape = (int(X.get_shape()[1]),1)
    w = tf.Variable(tf.random_normal(w_shape),name="weights")
    b = tf.Variable(0.0,name="bias")
    z = tf.add(tf.matmul(X,w),b,name="z")
    return tf.maximum(z,threshold,name="relu")

n_features = 3
X = tf.placeholder(tf.float32,shape=(None,n_features),name="X")
relus = [] 
for relu_index in range(5):
    with tf.variable_scope("relu",reuse=(relu_index>=1)) as scope:
        relus.append(relu(X))

output = tf.add_n(relus,name="output")

file_writer = tf.summary.FileWriter("logs/relu4", tf.get_default_graph())